In [1]:
### no brand names and search terms vector

In [11]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [12]:
%%time
%%cython
from gensim.models import doc2vec
import nltk
import re
def split_sentence (sentence):
    return re.split('\W+',sentence)
    
class MyDocs(object):
    def __iter__(self):
        for i, text in enumerate(open("txt_all_nb.txt")): # doesn't use brand name
            yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i])
            # Train the doc2vec model
cdef desp = MyDocs()
model = doc2vec.Doc2Vec(desp, size = 200, window = 8, min_count = 5, workers = 4)
model.save('distance_all.model')

CPU times: user 223 ms, sys: 570 ms, total: 793 ms
Wall time: 1.17 s


In [13]:
model = doc2vec.Doc2Vec.load('distance_all.model')

In [14]:
# Calculate the vector of search term according to our model
#def input_vec(x):
#    model.infer_vector(split_sentence(x))
# map(input_vec,open("txt_st.txt))
import numpy as np
pt_arrays = np.zeros((240760, 200))
for i,text in enumerate(open("txt_pt.txt")): # input search terms 
    pt_arrays[i]=model.infer_vector(split_sentence(text))   
pt_arrays

array([[ 0.02200266,  0.01908584,  0.00845405, ..., -0.00358907,
         0.00019054, -0.02040428],
       [ 0.06066574,  0.0153622 , -0.00077939, ...,  0.02107242,
        -0.00973943, -0.01212798],
       [ 0.03206712, -0.00783784, -0.00156679, ...,  0.02957773,
         0.00123606,  0.00124532],
       ..., 
       [ 0.06754978, -0.00714241, -0.01753183, ...,  0.02170464,
        -0.02912914,  0.03379539],
       [ 0.08341395,  0.01464552,  0.02370741, ...,  0.0169495 ,
        -0.05061054,  0.02032607],
       [ 0.01601431,  0.01445545,  0.01292743, ..., -0.00069231,
        -0.02125877,  0.02262981]])

In [15]:
import numpy as np
st_arrays = np.zeros((240760, 200))
for i,text in enumerate(open("txt_st.txt")): # input search terms 
    st_arrays[i]=model.infer_vector(split_sentence(text))   
st_arrays

array([[ 0.03758235, -0.08407412,  0.01892379, ...,  0.08160976,
        -0.01337415, -0.01347442],
       [ 0.04961559, -0.0095962 ,  0.00637948, ...,  0.06713269,
        -0.01257816, -0.00704337],
       [ 0.02635622,  0.0104771 , -0.01161926, ..., -0.01538065,
         0.00028396,  0.02770254],
       ..., 
       [-0.00763746, -0.03730408, -0.01154349, ...,  0.03627979,
         0.00559608,  0.03310749],
       [ 0.02992605,  0.04820156,  0.04666499, ...,  0.0133902 ,
         0.0107386 ,  0.00317763],
       [ 0.04723874, -0.01256068,  0.01744816, ...,  0.00654965,
         0.00027499, -0.01290956]])

In [16]:
## calculate the cosin distance between 2 vectors to generate the 3rd features
# Compute the cosine similarity values between the input text and all archived reviews
# cossims_with_input = map(lambda v: cossim(input_vec, v), model.docvecs) 
# need to chaneg the code into rows: calculate the cosin distance between the vectors in same rows
import numpy as np
import pandas as pd
# Calculate the cosine similarity between two vecotrs 
def cossim(v1, v2):
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2))

cossims_with_st=np.zeros((240760,1))
for i in range(240760):
    cossims_with_st[i] = cossim(st_arrays[i], pt_arrays[i])

In [17]:
np.savetxt("distance_all.csv",cossims_with_st, delimiter=",")

In [18]:
coswithst=pd.read_csv("distance_all.csv",header=None)
coswithst


,0
0,-0.039928
1,0.407894
2,0.214059
3,0.166730
4,0.437700
5,0.471011
6,0.180338
7,-0.054024
8,0.092775
9,0.338710


In [ ]:
cws_trn=coswithst.iloc[:num_train]
cws_tst=coswithst.iloc[num_train:]